# Experiment - Hyperparameter Tuning
* StellarAlgo Data Science
* January 20, 2023
* Grant Donst, Ryan Kazmerik, Peter Morrison

### This experiment is designed to test the effectiveness of hyper parameter tuning within the Pycaret package. We will first test the default configuration using RandomGridSearch and then test passing in a CustomGrid as well.

In [1]:
import pandas as pd

from data_sci_toolkit.aws_tools import redshift_tools
from pycaret.classification import *

/opt/miniconda3/envs/stellar/lib/python3.8/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [2]:
df = redshift_tools.get_retention_dataset(
    cluster= "prod-app",
    database= "stlrnhljets",
    lkupclientid= 92,
    start_year= 2018,
    end_year= 2021
)
df.shape

Authorized as AROASQ4JELIXYLYV6P4UV:rkazmerik@stellaralgo.com


(76645, 24)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76645 entries, 0 to 76644
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   lkupclientid         76645 non-null  int64         
 1   clientcode           76645 non-null  object        
 2   dimcustomermasterid  76645 non-null  int64         
 3   year                 76645 non-null  int64         
 4   productgrouping      76645 non-null  object        
 5   totalspent           76645 non-null  float64       
 6   recentdate           76645 non-null  datetime64[ns]
 7   attendancepercent    76645 non-null  float64       
 8   renewedbeforedays    76645 non-null  int64         
 9   source_tenure        76645 non-null  object        
 10  tenure               76645 non-null  int64         
 11  disttovenue          76645 non-null  float64       
 12  recency              76645 non-null  int64         
 13  missed_games_1       76645 non-

In [4]:
# copy original dataframe
df_dataset = df
features = [
    "attendancepercent",
    "disttovenue",
    "inperson_contact",
    "missed_games_1",
    "missed_games_2",
    "missed_games_over_2",
    "recency",
    "tenure",
    "totalspent",
    "isnextyear_buyer"
]

# create training and eval datasets
df_train = df_dataset.sample(frac=0.85, random_state=786)
df_train = df_train.reset_index(drop=True)

df_eval = df_dataset.drop(df_train.index)
df_eval = df_eval.reset_index(drop=True)

# print out the number of records for training and eval
print('Data for Training: ' + str(df_train.shape))
print('Data for Evaluation: ' + str(df_eval.shape), end="\n")

Data for Training: (65148, 24)
Data for Evaluation: (11497, 24)


In [5]:
setup(
    data = df_train, 
    target = 'isnextyear_buyer', 
    train_size = 0.85,
    data_split_shuffle = True,
    numeric_features = [
        "attendancepercent",
        "disttovenue",
        "inperson_contact",
        "missed_games_1",
        "missed_games_2",
        "missed_games_over_2",
        "recency",
        "tenure",
        "totalspent"
    ],
    silent = True,
    verbose = False
);

In [6]:
model_matrix = compare_models(
    fold=10,
    include= ["lightgbm"]
)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.8734,0.9433,0.7407,0.8041,0.7711,0.6838,0.6849,1.0100


In [7]:
best_model = create_model(model_matrix);

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8696,0.9416,0.7378,0.7946,0.7651,0.6751,0.6760
1,0.8736,0.9433,0.7415,0.8041,0.7715,0.6844,0.6854
2,0.8750,0.9421,0.7422,0.8081,0.7737,0.6876,0.6888
3,0.8738,0.9442,0.7396,0.8059,0.7713,0.6844,0.6856
4,0.8700,0.9404,0.7246,0.8043,0.7624,0.6732,0.6750
5,0.8698,0.9425,0.7282,0.8011,0.7629,0.6734,0.6749
6,0.8732,0.9458,0.7445,0.8008,0.7716,0.6840,0.6849
7,0.8716,0.9406,0.7465,0.7949,0.7700,0.6810,0.6817
8,0.8850,0.9489,0.7685,0.8205,0.7937,0.7140,0.7147


### Let's have a look at the parameters included in best_model:

In [8]:
plot_model(best_model, plot = 'parameter')

,Parameters
boosting_type,gbdt
class_weight,None
colsample_bytree,1.0
importance_type,split
learning_rate,0.1
max_depth,-1
min_child_samples,20
min_child_weight,0.001
min_split_gain,0.0
n_estimators,100


### Now let's auto tune the parameters, optimizing first for Accuracy, then AUC, then MCC:

In [9]:
acc_model = tune_model(best_model, optimize = 'Accuracy', n_iter=100)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8716,0.9420,0.7396,0.7993,0.7683,0.6797,0.6807
1,0.8731,0.9437,0.7384,0.8045,0.7700,0.6826,0.6838
2,0.8729,0.9423,0.7334,0.8073,0.7686,0.6812,0.6827
3,0.8738,0.9437,0.7353,0.8088,0.7703,0.6835,0.6850
4,0.8696,0.9407,0.7171,0.8083,0.7600,0.6709,0.6732
5,0.8707,0.9428,0.7232,0.8073,0.7629,0.6744,0.6763
6,0.8725,0.9460,0.7401,0.8015,0.7696,0.6816,0.6827
7,0.8694,0.9410,0.7409,0.7921,0.7656,0.6753,0.6760
8,0.8833,0.9490,0.7616,0.8203,0.7899,0.7093,0.7102


In [16]:
auc_model = tune_model(best_model, optimize = 'AUC', n_iter=120)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8713,0.9419,0.7371,0.7999,0.7672,0.6784,0.6795
1,0.8716,0.9435,0.7371,0.8010,0.7677,0.6792,0.6803
2,0.8747,0.9426,0.7302,0.8151,0.7704,0.6845,0.6865
3,0.8731,0.9447,0.7321,0.8087,0.7685,0.6814,0.6830
4,0.8667,0.9404,0.7177,0.7989,0.7561,0.6648,0.6666
5,0.8710,0.9433,0.7332,0.8016,0.7659,0.6772,0.6784
6,0.8743,0.9461,0.7420,0.8057,0.7725,0.6859,0.6870
7,0.8705,0.9406,0.7396,0.7961,0.7668,0.6774,0.6782
8,0.8794,0.9485,0.7535,0.8137,0.7824,0.6991,0.7001


In [17]:
mcc_model = tune_model(best_model, optimize = 'MCC', n_iter=120, )

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8691,0.9417,0.7378,0.7930,0.7644,0.6739,0.6747
1,0.8732,0.9434,0.7478,0.7989,0.7725,0.6848,0.6855
2,0.8740,0.9420,0.7403,0.8060,0.7717,0.6849,0.6861
3,0.8749,0.9431,0.7384,0.8100,0.7726,0.6865,0.6879
4,0.8669,0.9398,0.7183,0.7990,0.7565,0.6653,0.6671
5,0.8674,0.9417,0.7250,0.7960,0.7589,0.6677,0.6691
6,0.8716,0.9448,0.7363,0.8012,0.7674,0.6790,0.6801
7,0.8721,0.9408,0.7465,0.7965,0.7707,0.6822,0.6829
8,0.8839,0.9488,0.7685,0.8172,0.7921,0.7117,0.7123


### As we can see, tuning parameters using this method does not seem to increase performance on any of the targetted metrics :(

### Let's explore implementing a custom search method for hyperparameter tuning:

In [11]:
f1_model = tune_model(best_model, optimize='F1', search_library='scikit-optimize', )

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8671,0.9396,0.7396,0.7860,0.7621,0.6700,0.6706
1,0.8702,0.9422,0.7503,0.7884,0.7689,0.6787,0.6791
2,0.8711,0.9406,0.7359,0.8001,0.7667,0.6778,0.6789
3,0.8700,0.9419,0.7428,0.7925,0.7668,0.6768,0.6775
4,0.8682,0.9381,0.7365,0.7911,0.7628,0.6717,0.6725
5,0.8687,0.9404,0.7382,0.7914,0.7639,0.6731,0.6739
6,0.8691,0.9430,0.7470,0.7870,0.7665,0.6756,0.6761
7,0.8692,0.9386,0.7440,0.7896,0.7661,0.6755,0.6761
8,0.8801,0.9469,0.7679,0.8063,0.7866,0.7033,0.7037


### The results are showing that optimizing using this method actually decreases the model Accuracy :(

### Let's explore the alternate custom search library:

In [12]:
hype_model = tune_model(best_model, optimize='Accuracy', search_library='tune-sklearn', search_algorithm='hyperopt', n_iter=120)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8716,0.9418,0.7302,0.8055,0.7660,0.6779,0.6794
1,0.8741,0.9436,0.7346,0.8104,0.7706,0.6842,0.6858
2,0.8758,0.9423,0.7321,0.8172,0.7723,0.6873,0.6892
3,0.8761,0.9439,0.7353,0.8162,0.7736,0.6887,0.6904
4,0.8666,0.9407,0.7083,0.8047,0.7534,0.6625,0.6650
5,0.8709,0.9432,0.7163,0.8127,0.7614,0.6734,0.6759
6,0.8729,0.9463,0.7338,0.8068,0.7686,0.6812,0.6826
7,0.8741,0.9410,0.7403,0.8066,0.7720,0.6853,0.6865
8,0.8830,0.9492,0.7578,0.8218,0.7885,0.7078,0.7089


### We see very modest gains in almost every metric, and wonder if training on SageMaker with a much larger instances would allow us to increase the number of iterations to 10,000 and see more significant improvements.